In [1]:
import pandas as pd
import requests
import gzip
import json
import io

Read json file in

In [4]:
url = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Vermont_10.json.gz'

response = requests.get(url, stream = True)
response.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response.content), mode = 'rb') as gz_file:
    data_list = [json.loads(line) for line in gz_file]

vt = pd.DataFrame(data_list)

In [5]:
print(vt.shape)
print(vt.columns)
print(vt.head())

(324725, 8)
Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')
                 user_id             name           time  rating  \
0  118026874392842649478    rebecca kerns  1620085852324       5   
1  101532740754036204131    Peter DeForge  1580309946474       5   
2  115404122636203550540    Chad Goulette  1605195974445       5   
3  104789336434407408181  Mark LaFountain  1593005848256       5   
4  108980665975608069965           Jeff R  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'time': 1620087641504, 'text': 'Good Evening,...  

Drop 'name' column, convert all text to lower case, drop duplicates (~3000 duplicates)

In [54]:
vt = vt.drop(['name'], axis = 1)
vt.columns = vt.columns.str.lower() 
vt = vt.drop_duplicates(subset=['user_id', 'text', 'gmap_id'])
print(vt.shape)

(321483, 7)


Change the format of the 'pics' column, merge multiple dictionaries into one with list of pic urls

In [58]:
def collapse_pics(pic_list):
    if not pic_list:
        return []  
    urls = []
    for pic_dict in pic_list:
        urls.extend(pic_dict.get('url', []))
    return urls

vt['pics_collapsed'] = vt['pics'].apply(collapse_pics)

Create column for average star rating for each business

In [60]:
average_ratings = vt.groupby('gmap_id')['rating'].mean()
vt['average_rating'] = vt['gmap_id'].map(average_ratings)
print(vt.head())

        user_id           time  rating  \
0  1.180269e+20  1620085852324       5   
1  1.015327e+20  1580309946474       5   
2  1.154041e+20  1605195974445       5   
3  1.047893e+20  1593005848256       5   
4  1.089807e+20  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'time': 1620087641504, 'text': 'Good Evening,...   
1  {'time': 1580320228721, 'text': 'Good Afternoo...   
2  {'time': 1605195166792, 'text': 'Hi Chad!

Tha...   
3  {'time': 1593376422014, 'text': 'Mark, thank y...   
4  {'time': 1582063833737, 'text': 'Good Afternoo...   

                                 gmap

Separate dataset into vt1 (rating only) and vt2 (everything else)

In [59]:
vt1 = vt[vt['pics'].isna() & vt['text'].isna()] # rating only
vt2 = vt[~vt.index.isin(vt1.index)] # everything else

print(vt1.shape)
print(vt2.shape)

(145565, 8)
(175918, 8)


Data Exploration

In [6]:
print(vt)

                      user_id             name           time  rating  \
0       118026874392842649478    rebecca kerns  1620085852324       5   
1       101532740754036204131    Peter DeForge  1580309946474       5   
2       115404122636203550540    Chad Goulette  1605195974445       5   
3       104789336434407408181  Mark LaFountain  1593005848256       5   
4       108980665975608069965           Jeff R  1582059996120       5   
...                       ...              ...            ...     ...   
324720  102604405041956328019   Nicole Couture  1534560336677       5   
324721  113667429629445112279      Holly Chase  1601130739739       5   
324722  101031262113246567699  Jennie Drummind  1503368927511       5   
324723  113511800890575185422     John Hawkins  1566750149101       5   
324724  107697312145259650604            Mindy  1521208726615       5   

                                                     text  pics  \
0           Always done right from wood stove to screens